# Advanced Modeling and Simulation Course

### Instructor: Dr. M. Nedim ALPDEMİR

*This material is prepared as part of the EE592-Advanced Modeling And Simulation course given at Yıldırım Beyazıd University (YBÜ).*

# Simulation of an Inventory System

## Problem Statement

* A company that sells a single product would like to decide how many items it should have in inventory for each of the next n months. 
* The times between demands are IID (Independent and Identically Distributed) **exponential random variables** with a mean of 0.1 month. 
* The sizes of the demands, D, are IID **random variables** (independent of when the demands occur), with:

$$
Z = \Bigg \{
\begin{array}{ccc} 
1 & w.p. & 1/6 \\
2 & w.p. & 1/3 \\
3 & w.p. & 1/3 \\
4 & w.p. & 1/6 
\end{array}
$$ 

where w.p. stands for _with probability_

* At the beginning of each month, the company reviews the inventory level and decides how many items to order from its supplier. 
* If the company orders Z items, it incurs a cost of $K + iZ$, where $K = \$32$ is the setup cost and $i = \$3$ is the incremental cost per item ordered. (If Z = 0, no cost is incurred.) 
* When an order is placed, the time required for it to arrive (called the delivery lag or lead time) is **a random variable** that is distributed **uniformly** between 0.5 and 1 month. i.e., The company uses a stationary $(s,S)$ policy to decide how much to order. İ.e.,

$$
Z = \Bigg \{
\begin{array}{ccc} 
S-I & if & I<s \\
0 & if & I>=s
\end{array}
$$ 

where $I$ is the inventory level at the beginning of the month.

* When a demand occurs, it is satisfied immediately if the inventory level is at least as large as the demand. 
* If the demand exceeds the inventory level, theexcess of demand over supply is backlogged and satisfied by future deliveries. *(In this case, the new inventory level is equal to the old inventory level minus the demand size, resulting in a negative inventory leveL) *
* When an order arrives, it is first used to eliminate as much of the backlog (if any) as possible; the remainder of the order (if any) is added to the inventory. 

Also;

* Let let $I(t)$  be the inventory level at time $t$. 
* let $I^+ (t) = max\{ l(t), O \}$ be the number of items physically on hand in the inventory at time $t$  
* and let $I^- (t) = max\{ -l(t), O \}$ be the backlog at time $t$ 


For our model, 

* we shall assume that the company incurs a **_holding cost_** (rental,insurance, taxes, maintenance etc) of $h = \$1$ per item per month held in (positive) inventory.  
* since $I^+ (t)$ is the number of items held in inventory at time $t$, the time-average
(per month) number of items held in inventory for the n-month period is:

$$ {I}^+_{avg} = \frac{\int\limits_0^n I^+ (t) \ dt} {n}$$

* Thus, average holding cost per month is $h \times {I}^+_{avg}$

Similarly;

* suppose that the company incurs a backlog cost of $ k= \$5 $ per item per month in backlog; this accounts for the cost of extra'record keeping when a backlog exists, as well as loss of customers' goodwill. 
* The time-average number of items in backlog is:

$$ {I}^-_{avg} = \frac{\int\limits_0^n I^- (t) \ dt} {n}$$

* So, average shortage cost per month is $k \times {I}^-_{avg}$


## Our Objective for the Simulation


* Assume that the initial inventory level is $I(0) = 60$ and that no order is outstanding. 
* We simulate the inventory system for n = 120 months and use the average total cost per month (which is the sum of the average ordering cost per month, the average holding cost per month, and the average shortage cost per month) to compare the following nine inventory policies:


|Label                   | p1 | p2 | p3 | p4  | p5 | p6 | p7  | p8 | p9  |                       
|------------------------|----|----|----|-----|----|----|-----|----|-----|
|Min inventory level (s) | 20 | 20 | 20 | 20  | 40 | 40 | 40  | 60 | 60  |
|Max inventory level (S) | 40 | 60 | 80 | 100 | 60 | 80 | 100 | 80 | 100 |


## Determine the event types


There are 4 event types in this system

|Event Description                                                        |  Event Type  |    
|-------------------------------------------------------------------------|--------------|
|Arrival of an order to the company from the supplier                     |       1      | 
|Demand for the product form a customer                                   |       2      | 
|End of the simulation after n months                                     |       3      | 
|Inventory evaluation (end possible ordering) at the beginning of a month |       4      | 

## Draw the Event Graphs

* In an event graph events, each represented by a node, are connected by directed arcs (arrows) depicting how events may be scheduled from other events and from themselves 

* For example, in the Inventory simulation event graph given below, the evaluate event schedules another future occurrence of itself and (possibly) an order arrival event, and the demand event may schedule another future occurrence of itself; 
* in addition, the demand event must be initially scheduled in order to get the simulation going. 



![inventoryGraph](InventoryEventGraph.png)


## Draw the event processing flow charts if necessary

* Normally you would want to design one process for each event type
* The `End Simulation` Event is a termination event so no need to implement a separate process. 
* So we have three processes namely:
    - **Order Processor**
    - **Customer Demand Generator**
    - **Inventory Control**
* it is a good practice to draw flow charts or write some pseudo code to describe, roughly, the implementation logic of those processes


![EventFlow-1](EventFlow-1.png)


![InventoryEventFlow-1](InventoryEventFlow-2.png)

## Input Data Modeling

There are three types of random variates needed to simulate this system:

1. The interdemand times are distributed exponentially,  
2. The demand-size random variate $D$ must be discrete, as described above, and can be generated using weighted uniform random generator. 4 weights $[1/6, 1/3, 1/3, 1/6]$ will be assigned to four values $[1, 2, 3, 4]$ 
3. The delivery lags are uniformly distributed, but not over the unit interval $[0,1]$. In general, we can generate a random variate distributed uniformly over any interval $[a, b]$. So in this case the interval is $[0.5, 1.0]$ in months


In [1]:
import itertools
import random
import numpy as np
import simpy

# INPUT PARAMETERS 
RANDOM_SEED = 40
DEMAND_SIZES = [1, 2, 3 ,4]   # These are the possible values for variabie D (i.e. demand size) 
DEMAND_WEIGHTS = [1/6, 1/3, 1/3, 1/6] # probability of each possible demand size
INT_DEMAND_TIME_MEAN = 0.1     # mean value for interdemand times. Used to generate expovariate
DELIVERY_LAG = [0.5, 1.0]        # Min/max values for delivery lag - in months
SIM_TIME = 120                # Simulation time in months
POLICIES = [(20,40), (20,60),(20,80), (20,100),(40,60),(40,80),(40,100),(60,80),(60,100)]
#POLICIES = [(20,40), (40,100)]
UNIT_BACKLOG_COST = 5        # backlog cost if a customer demand wasn't met (in dollars) 
ORDER_SETUP_COST =  32       # setup cost for a new order (in dollars)
UNIT_INCR_COST = 3           # incremental cost per item ordered (in dollars)
UNIT_HOLDING_COST = 1        # unit holding cost (in dollars)
INIT_INVENTORY_LEVEL = 60
INVENTORY_CAPACITY = 220

# MODEL (INTERNAL) VARIABLES
class SimulationState:
    def __init__(self):
        self.T_BACKLOG_LEVEL = 0
        self.CUR_BACKLOG_LEVEL = 0
        self.T_HOLDING_COST = 0
        self.T_BACKLOG_COST = 0
        self.T_ORDER_COST = 0
        self.LAST_INV_LEVEL_CHK_TIME = 0
    def reset_state(self):
        self.T_BACKLOG_LEVEL = 0
        self.CUR_BACKLOG_LEVEL = 0 # current backlog level. this is a more transient state since it can be more frequently reset
        self.T_HOLDING_COST = 0
        self.T_BACKLOG_COST = 0
        self.T_ORDER_COST = 0
        self.LAST_INV_LEVEL_CHK_TIME = 0
# OUTPUT VARIABLES
AVG_TOTAL_COST = 0        # Average total cost per month
AVG_HOLDING_COST = 0      # Average holding cost per month
AVG_ORDER_COST = 0        # Average ordering cost per month
AVG_SHORT_COST = 0        # Average shortage cost per month 


In [2]:
# helper classes
# implement a logger
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)
logging.getLogger().setLevel('INFO')

In [15]:
def customer_demand_generator(env, product_stock, state):
    """Generate new demands."""
    while True:
        yield env.timeout(np.random.exponential(INT_DEMAND_TIME_MEAN))
        #yield env.timeout(random.expovariate(1/INT_DEMAND_TIME_MEAN))
        demand_size = np.random.choice(DEMAND_SIZES, p=DEMAND_WEIGHTS)
        logging.debug(msg = "demand for %d units"% demand_size)
        try: 
            #get current stock level
            current_stock_level = product_stock.level
            if demand_size > current_stock_level:
                state.T_BACKLOG_LEVEL +=  demand_size - current_stock_level
                state.CUR_BACKLOG_LEVEL += demand_size - current_stock_level
                logging.debug(msg = "total back log = %d"%state.T_BACKLOG_LEVEL)
                logging.debug(msg = "current back log = %d"%state.CUR_BACKLOG_LEVEL)
                if current_stock_level > 0:
                    product_stock.get(current_stock_level)
            else:
                logging.debug("getting from the stock. Amount : %d"%demand_size)
                product_stock.get(demand_size)
                logging.debug("Now stock: %d"%product_stock.level)
        except ValueError:
            print("Value Error ProductStock level = %d"%current_stock_level)
        

In [16]:
def inventory_control(env, current_policy, product_stock, state):
    """ Check for stock level and schedule orders"""
    while True:
        yield env.timeout(1.0);
        min_inv_level = current_policy[0]
        max_inv_level = current_policy[1]
        hold_time = env.now - state.LAST_INV_LEVEL_CHK_TIME
        state.LAST_INV_LEVEL_CHK_TIME = env.now
        state.T_HOLDING_COST += product_stock.level * hold_time
        state.T_BACKLOG_COST += state.T_BACKLOG_LEVEL * hold_time 
        state.T_BACKLOG_LEVEL = 0
        if product_stock.level < min_inv_level:
            # stock level below threshold so initiate an order cost
            env.process(order_processor(env, product_stock, current_policy, state))
                
        
    

In [17]:
def order_processor(env, product_stock, policy, state):
    """ This process handles the case where Inventory is low on stock schedules an order arrival event"""
    # generate a randow variate for order arrival. 
    # This event is modeled to be generated from a uniform distribution
    order_arrival_time = np.random.uniform(DELIVERY_LAG[0], DELIVERY_LAG[1])
    yield env.timeout(order_arrival_time)
    # new order has arrived 
    # calculate the ordered amount in accordance with the current policy
    order_amount = policy[1] - product_stock.level
    state.T_ORDER_COST += ORDER_SETUP_COST + order_amount * UNIT_INCR_COST   
    if state.CUR_BACKLOG_LEVEL > 0:
        stocking_amount = order_amount - state.CUR_BACKLOG_LEVEL
        if stocking_amount > 0:
            # if the ordered amount is more than the backlog 
            # then clear the backlog and add the excess to the inventory
            logging.debug(msg = "clearing backlog required,  putting stocking amount = %d"%stocking_amount)
            product_stock.put(stocking_amount)
            logging.debug("addding to the stock. Stock level now = %d"%product_stock.level)
            logging.debug("resetting backlog level to 0")
            state.CUR_BACKLOG_LEVEL = 0
        else:
            # insufficient units ordered 
            # still have uncleared backlog, and stock level is still 0 (empty)
            state.CUR_BACKLOG_LEVEL = state.CUR_BACKLOG_LEVEL - order_amount
    else:
        # there is no backlog to clear so put the full order amount into the stock
        logging.debug("no backlog. Stock full order amount = %d"%order_amount)
        product_stock.put(order_amount)


In [42]:
# Setup and start the simulation
logging.info('Inventory Simulation Results')
random.seed(RANDOM_SEED)

SimState = SimulationState() 
results = []
logging.info("{0:16s} {1:16s} {2:16s} {3:16s} {4:16s}".format(" policy", "AVG_TOTAL_COST", "AVG_HOLDING_COST", "AVG_SHORT_COST", "AVG_ORDER_COST"))
for policy in POLICIES:
    # Create environment and start processes
    env = simpy.Environment()
    product_stock_depo = simpy.Container(env, INVENTORY_CAPACITY, init=INIT_INVENTORY_LEVEL)
    env.process(customer_demand_generator(env, product_stock_depo, SimState))
    env.process(inventory_control(env, policy, product_stock_depo, SimState))    
    # Execute!
    env.run(until=SIM_TIME)
    AVG_HOLDING_COST = UNIT_HOLDING_COST * SimState.T_HOLDING_COST / SIM_TIME
    AVG_SHORT_COST = UNIT_BACKLOG_COST * SimState.T_BACKLOG_COST / SIM_TIME
    AVG_ORDER_COST = SimState.T_ORDER_COST / SIM_TIME
    AVG_TOTAL_COST = AVG_HOLDING_COST + AVG_ORDER_COST + AVG_SHORT_COST
    results.append([policy, AVG_TOTAL_COST, AVG_HOLDING_COST, AVG_SHORT_COST, AVG_ORDER_COST])
    logging.info ("{0:16s} {1:14.2f} {2:14.2f} {3:14.2f} {4:14.2f}".format(str(policy), AVG_TOTAL_COST, AVG_HOLDING_COST, AVG_SHORT_COST, AVG_ORDER_COST))
    SimState.reset_state()

INFO:Inventory Simulation Results
INFO: policy          AVG_TOTAL_COST   AVG_HOLDING_COST AVG_SHORT_COST   AVG_ORDER_COST  
INFO:(20, 40)                  65.40          24.30           0.00          41.10
INFO:(20, 60)                  73.70          31.00           4.50          38.20
INFO:(20, 80)                  62.50          40.70           0.00          21.80
INFO:(20, 100)                 83.80          51.80           0.00          32.00
INFO:(40, 60)                  81.00          39.90           0.00          41.10
INFO:(40, 80)                  91.40          46.60           0.00          44.80
INFO:(40, 100)                107.90          56.20           0.00          51.70
INFO:(60, 80)                 111.60          56.80           0.00          54.80
INFO:(60, 100)                105.40          68.70           0.00          36.70


In [44]:
logging.info('Inventory Simulation Results')
random.seed(RANDOM_SEED)

SimState = SimulationState() 

logging.info("{0:16s} {1:16s} {2:16s} {3:16s} {4:16s}".format(" policy", "ExpMean_TOTAL_COST", "Conf_interval", "min_interval", "max_interval"))
REPLICATION_NUM = 10
for policy in POLICIES:
    EXP_MEAN = 0
    TOTAL = 0
    VARIANCE = 0
    MIN = 0
    MAX = 0
    SUM = 0
    results = []
    for i in range(REPLICATION_NUM):
        # Create environment and start processes
        env = simpy.Environment()
        product_stock_depo = simpy.Container(env, INVENTORY_CAPACITY, init=INIT_INVENTORY_LEVEL)
        env.process(customer_demand_generator(env, product_stock_depo, SimState))
        env.process(inventory_control(env, policy, product_stock_depo, SimState))    
        # Execute!
        env.run(until=SIM_TIME)
        AVG_HOLDING_COST = UNIT_HOLDING_COST * SimState.T_HOLDING_COST / SIM_TIME
        AVG_SHORT_COST = UNIT_BACKLOG_COST * SimState.T_BACKLOG_COST / SIM_TIME
        AVG_ORDER_COST = SimState.T_ORDER_COST / SIM_TIME
        AVG_TOTAL_COST = AVG_HOLDING_COST + AVG_ORDER_COST + AVG_SHORT_COST
        TOTAL += AVG_TOTAL_COST

        results.append(AVG_TOTAL_COST)
        SimState.reset_state()
    EXP_MEAN = TOTAL / REPLICATION_NUM
    for t in results:
        #print(t, EXP_MEAN)        
        SUM += (t - EXP_MEAN)**2
    VARIANCE = SUM / (REPLICATION_NUM  - 1)
    CONF_INTERVAL = 1.812*np.sqrt(VARIANCE / REPLICATION_NUM)
    MIN = EXP_MEAN - CONF_INTERVAL
    MAX = EXP_MEAN + CONF_INTERVAL
    logging.info ("{0:16s} {1:14.2f} {2:14.2f} {3:14.2f} {4:14.2f}".format(str(policy), EXP_MEAN, CONF_INTERVAL, MIN, MAX))


INFO:Inventory Simulation Results
INFO: policy          ExpMean_TOTAL_COST Conf_interval    min_interval     max_interval    
INFO:(20, 40)                  60.57           3.54          57.03          64.11
INFO:(20, 60)                  70.28           4.09          66.19          74.37
INFO:(20, 80)                  75.10           7.17          67.93          82.27
INFO:(20, 100)                 84.44           4.44          80.00          88.88
INFO:(40, 60)                  82.50           1.41          81.09          83.91
INFO:(40, 80)                  93.84           4.57          89.27          98.41
INFO:(40, 100)                105.22           7.00          98.22         112.22
INFO:(60, 80)                 106.29           3.63         102.66         109.92
INFO:(60, 100)                116.48           5.21         111.27         121.69
